<a href="https://colab.research.google.com/github/AWH-GlobalPotential-X/AWH-Geo/blob/main/Population_byOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tool runs a cumulative histogram of the population without SMDW across output in 0.5 L/day intervals by device, material, or characteristic specific yield curve.

This tool requires a Google Drive and Earth Engine Account

Click "Connect" at the top right of this notebook.

Then run each of the code blocks below, following instructions.

In [ ]:
#@title Basic setup and earthengine access.

print('Welcome to the Population SMDW by Output Tool')

# import, authenticate, then initialize EarthEngine module ee
# https://developers.google.com/earth-engine/python_install#package-import
import ee 
print('Make sure the EE version is v0.1.215 or greater...')
print('Current EE version = v' + ee.__version__)
print('')
ee.Authenticate()
ee.Initialize()

worldGeo = ee.Geometry.Polygon( # Created for some masking and geo calcs
  coords=[[-180,-90],[-180,0],[-180,90],[-30,90],[90,90],[180,90],
          [180,0],[180,-90],[30,-90],[-90,-90],[-180,-90]],
  geodesic=False,
  proj='EPSG:4326'
)



In [ ]:
#@title Click to run calculation

"""
This script runs through each of the output images (in average L/d) created in AWH-Geo
of real AWH devices, upper limits, and characteristic curves and exports a cumulative
histogram of the population without SMDW that lives within each interval of 0.5 L/d.
This graphs the number of population whos output is at least X L/d. 

This is to produce figure 3c in the Global Potential for Harvesting Drinking 
Water from Air paper.
""" 

# Overall mean output (L/d) image data import
# Real devices
Bagheri2018 = ee.Image('projects/h2e/results/Od_overallMean_Bagheri2018').set('name','Bagheri2018')
ZMW = ee.Image('projects/h2e/results/Od_overallMean_ZMW_SOURCE').set('name','ZMW')

# Experimental sorbent material
SMAG_Zhao = ee.Image('projects/h2e/results/Od_overallMean_SMAG_Zhao').set('name','SMAG_Zhao')

# Theoretical limits
CondenserPeeters = ee.Image('projects/h2e/results/Od_overallMean_CoolerCondensorWithoutRecup_PeetersFigure11').set('name','CondenserPeeters')
Kim2020 = ee.Image('projects/h2e/results/Od_overallMean_Kim2020Limits_100C_20201013').set('name','Kim2020')

# Characteristic curves for target user base
# Logistic
charCurve_target1500mil_logistic = ee.Image('projects/h2e/results/Od_overallMean_logisticB').set('name','charCurve_target1500mil_logistic')
charCurve_target1000mil_logistic = ee.Image('projects/h2e/results/Od_overallMean_logisticE').set('name','charCurve_target1000mil_logistic')
charCurve_target500mil_logistic = ee.Image('projects/h2e/results/Od_overallMean_logisticD').set('name','charCurve_target500mil_logistic')

# Linear
charCurve_target1500mil_linear = ee.Image('projects/h2e/results/Od_overallMean_linearB').set('name','charCurve_target1500mil_linear')
charCurve_target1000mil_linear = ee.Image('projects/h2e/results/Od_overallMean_linearE').set('name','charCurve_target1000mil_linear')
charCurve_target500mil_linear = ee.Image('projects/h2e/results/Od_overallMean_linearC').set('name','charCurve_target500mil_linear')

# import popultation without SMDW
popWtrSt = ee.Image('projects/h2e/publication/pop_wtrSt')
scale = popWtrSt.projection().nominalScale()

# organize images into a collection
collection = ee.ImageCollection([
  
  Bagheri2018,
  CondenserPeeters,
  Kim2020,
  SMAG_Zhao,
  ZMW,
  
  charCurve_target1500mil_logistic,
  charCurve_target1000mil_logistic,
  charCurve_target500mil_logistic,
  
  charCurve_target1500mil_linear,
  charCurve_target1000mil_linear,
  charCurve_target500mil_linear
  
  ])


def seqSum(i):
  
  # this function runs sequential summations of population within each 0.5 L/day 
  # bin of the mean output images
  
  i = ee.Image(i)
  
  outputThresholds = ee.List.sequence(0,20,0.5).cat([25,30,50,70,100]) # list bins as thresholds

  def reduceByThresh(threshold):
    
    # spatial calculation of population sum within threshold band

    threshold = ee.Number(threshold)

    def ifNulls(l): # nested if statements to null out bins without 0's, 1's, or both
      l = ee.Dictionary(l)
      return ee.Algorithms.If(
        condition=ee.Number(l.get('threshold')).eq(1),
        trueCase=l.get('sum'),
        falseCase=None
      )

    pop_atThreshold = ee.List(popWtrSt.addBands(i.gte(threshold # binary image, 1=over threshold
                                                      )).reduceRegion(
      reducer=ee.Reducer.sum().unweighted().group(1,'threshold'), # run reducer, grouped by binary class
      geometry=worldGeo, 
      scale=scale,
      maxPixels=1e12
    ).get('groups')).map(ifNulls, True)

    return ee.Algorithms.If(
      condition=pop_atThreshold.size().eq(0),
      trueCase=ee.Number(0),
      falseCase=ee.Number(pop_atThreshold.get(0))
    );    

  pop_byOutput = outputThresholds.map(reduceByThresh) # run threshold values over reducer
  
  return ee.Feature( # return results as feature collection without geometries (i.e. a table)
    None, ee.Dictionary({'0': i.get('name')}).combine(
      ee.Dictionary.fromLists(
        keys=outputThresholds.map(lambda l: ee.Number(l).multiply(100).int().format('%05d')),
        values=pop_byOutput.map(lambda l: ee.Number(l))
        ))
    )

# run calculation across all images
results = ee.FeatureCollection(collection.map(seqSum))

# Export to drive
task = ee.batch.Export.table.toDrive(
  collection=results,
  # folder='', # write Drive folder here if desired
  description='PopSMDW_byOutput_fig3c',
  fileNamePrefix='PopSMDW_byOutput_fig3c',
  fileFormat='CSV'
)
task.start()
print('Exporting PopSMDW_byOutput_fig3c')
print('Complete')




Go to https://code.earthengine.google.com/ to track task progress. 

CSV file will land in Drive.